<a href="https://colab.research.google.com/github/AdamJelley/AdventOfCode2021/blob/main/Day18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 18

In [2]:
import requests
import numpy as np

In [3]:
#Variables to set
day = 18
cookie = ''

In [4]:
input_link = f'https://adventofcode.com/2021/day/{day}/input'
user_cookie = {'session':cookie} # Retrieve session cookie corresponding to user login (by inspecting cookies on data page)

In [5]:
raw_input = requests.get(input_link, cookies=user_cookie).text[:-1]
list_input = raw_input.split('\n')
processed_input = [eval(snum) for snum in list_input]

In [6]:
def parse_snum(snum):
  snum_char_list = list(str(snum))
  for i, char in enumerate(snum_char_list):
    if char.isnumeric() and snum_char_list[i+1].isnumeric():
      snum_char_list[i:i+2] = [''.join(snum_char_list[i:i+2])]
  snum_list = [int(e) if e.isnumeric() else e for e in snum_char_list]
  return snum_list

In [7]:
def unparse_snum(snum):
  snum_str = ''.join([str(e) for e in snum])
  return snum_str

## Part 1

In [8]:
def snum_add(snum1, snum2):
  snum_sum = f'[{snum1}, {snum2}]'
  return snum_sum

In [9]:
def explode_pair(snum):
  explode = False
  depth=0
  for i, char in enumerate(snum):
    depth=depth+1 if char=='[' else depth
    depth=depth-1 if char==']' else depth
    
    if depth==5:
      explode=True
      for j in range(i,0,-1):
        if type(snum[j])==int:
          snum[j]+=snum[i+1]
          break
      for k in range(i+5, len(snum)):
        if type(snum[k])==int:
          snum[k]+=snum[i+4]
          break
      
      del snum[i:i+6]
      snum.insert(i, 0)
      break
    
    
  #print(''.join([str(e) for e in snum]))
  return snum, explode

In [10]:
def split_num(snum):
  split=False
  for i, char in enumerate(snum):
    if type(char)==int:
      if char>9:
        split=True
        first = char//2
        second = char//2 if char%2==0 else char//2+1
        del snum[i]
        snum[i:i] = f'[ ,  ]'
        snum[i+1]=first
        snum[i+4]=second
        break
  return snum, split

In [11]:
def reduce_snums(snum1, snum2):
  snum_sum = snum_add(snum1, snum2)
  snum_sum = parse_snum(snum_sum)
  explode = True
  split=True
  while explode==True or split==True:
    explode=True
    while explode==True:
      snum_sum, explode = explode_pair(snum_sum)
    snum_sum, split = split_num(snum_sum)
  snum_sum_str = unparse_snum(snum_sum)
  return snum_sum_str

In [12]:
def add_reduce_snums(snums):
  result = snums[0]
  for snum in snums[1:]:
    result = reduce_snums(result, snum)
  return result

In [27]:
def calculate_magnitude(snum):
  parsed_snum=parse_snum(snum)
  while '[' in parsed_snum:
    old_parsed_snum = parsed_snum.copy()
    for i, char in enumerate(old_parsed_snum[:-3]):
      if type(char)==int:
        if type(parsed_snum[i+3])==int: 
          magnitude=3*int(char)+2*int(parsed_snum[i+3])
          del parsed_snum[i:i+5]
          parsed_snum[i-1]=magnitude
          break

  return parsed_snum[0]

In [28]:
test_input = """[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
[7,[5,[[3,8],[1,4]]]]
[[2,[2,2]],[8,[8,1]]]
[2,9]
[1,[[[9,3],9],[[9,0],[0,7]]]]
[[[5,[7,4]],7],1]
[[[[4,2],2],6],[8,7]]"""

In [29]:
processed_test_input = [eval(snum) for snum in test_input.split('\n')]

In [30]:
add_reduce_snums(processed_test_input)==str([[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]])

True

In [31]:
test_result = add_reduce_snums(processed_test_input)
test_magnitude = calculate_magnitude(test_result)
print(test_magnitude)

3488


In [32]:
result = add_reduce_snums(processed_input)
magnitude = calculate_magnitude(result)
print(f'Resulting snum from snum addition of input: {result}\nWith magnitude: {magnitude}')

Resulting snum from snum addition of input: [[[[9, 8], [6, 0]], [[7, 6], 8]], [[[3, 2], 5], 8]]
With magnitude: 2501


## Part 2

In [33]:
snum_sums = {}
for snum1 in processed_input:
  for snum2 in processed_input:
    result = reduce_snums(snum1, snum2)
    magnitude = calculate_magnitude(result)
    snum_sums[(str(snum1), str(snum2))] = magnitude

In [35]:
snums, magnitude = max(snum_sums.items(), key=lambda x:x[1])
print(f'Max magnitude = {magnitude}\nFrom snums: {snums}')

Max magnitude = 4935
From snums: ('[[[9, [6, 6]], [[4, 8], [1, 3]]], [[[4, 7], 8], [[5, 2], [3, 8]]]]', '[[9, [[9, 3], [3, 7]]], [[[6, 9], 1], [[2, 3], [4, 4]]]]')
